# Resource project scheduling problems

## Learning Objectives

By the end of this notebook, you will be able to:

* **Define** a Resource Constrained Project Scheduling Problem (RCPSP) and its key components.
* **Load and inspect** a standard scheduling problem instance using the `discrete-optimization` library.
* **Visualize** the problem's precedence constraints as a network graph.
* **Understand and compute** the Critical Path to find a lower bound for the project's duration.
* **Implement** a fundamental scheduling heuristic: the Serial Schedule Generation Scheme (SGS).
* **Evaluate** the quality of a generated schedule by its makespan.

## RCPSP definition
The problem is made of $M$ activities that have precedence constraints. That means that if activity $j\in[1,M]$ is a successor of activity $i\in[1,M]$, then activity $i$ must be completed before activity $j$ can be started

On top of these constraints, each project is assigned a set of K renewable resources where each resource $k$ is available in $R_{k}$ units for the entire duration of the project. Each activity may require one or more of these resources to be completed. While scheduling the activities, the daily resource usage for resource $k$ can not exceed $R_{k}$ units.

Each activity $j$ takes $d_{j}$ time units to complete.

The overall goal of the problem is usually to minimize the makespan.

A classic variant of RCPSP is the multimode RCPSP where each task can be executed in several ways (one way=one mode). A typical example is :

Mode n°1 'Fast mode': high resource consumption and fast
Mode n°2 'Slow mode' : low resource consumption but slow

## Prerequisites

Before running this notebook, you need to 
- install discrete-optimization in your jupyter kernel
    ```
    pip install discrete-optimization
    ```

In [ ]:
#!pip install discrete_optimization # should take 2 minutes ?

## Imports

In [ ]:
import os
os.environ["DO_SKIP_MZN_CHECK"] = "1"
from typing import List, Hashable
import logging
import random
import matplotlib.pyplot as plt
import nest_asyncio
import networkx as nx
import numpy as np
from discrete_optimization.datasets import fetch_data_from_psplib
from discrete_optimization.rcpsp.problem import RcpspProblem, RcpspSolution
from discrete_optimization.rcpsp.parser import get_data_available, parse_file
from discrete_optimization.rcpsp.utils import (
    Graph,
    compute_graph_rcpsp,
    plot_ressource_view,
    plot_task_gantt,
)
# patch asyncio so that applications using async functions can run in jupyter
nest_asyncio.apply()
# set logging level
logging.basicConfig(level=logging.INFO)

### Download datasets

If not yet available, we import the datasets from [psplib](https://www.om-db.wi.tum.de/psplib/data.html).

In [ ]:
needed_datasets = ["j301_1.sm"]
download_needed = False
try:
    files_available_paths = get_data_available()
    for dataset in needed_datasets:
        if len([f for f in files_available_paths if dataset in f]) == 0:
            download_needed = True
            break
except:
    download_needed = True

if download_needed:
    fetch_data_from_psplib()

## Loading the problems definition

In [ ]:
files_available = get_data_available()

Now we can load some RCPSP problem from provided examples

In [ ]:
filepath = [f for f in get_data_available() if "j301_1.sm" in f][0]
with open(filepath, "rt") as f:
    print(f.read())

There are 32 jobs, including the source task (1) and the sink task (32). 

- The first part of the file describe the precedence constraints : 
  > Task $1$ should finish before task $2$, $3$, $4$ start.
  
- The second part of the file details the duration and resource usage per task : 
  > Task $3$ lasts 4 units of times and requires 10 units of $R_1$

### Parsing file
We parse the file to get a RCPSP model object in discrete-optimization library.

In [ ]:
filepath = [f for f in get_data_available() if "j301_1.sm" in f][0]
rcpsp_problem = parse_file(filepath)
print(type(rcpsp_problem))
print(rcpsp_problem)
print("Nb jobs : ", rcpsp_problem.n_jobs)
print("Precedences : ", rcpsp_problem.successors)
print("Resources Availability : ", rcpsp_problem.resources)

### Precedence graph
We can have a visual version of the precedence graph :

In [ ]:
def plot_predecessors_graph(rcpsp_problem: RcpspProblem, 
                            path: list[Hashable] = None):
    graph_nx = rcpsp_problem.graph.to_networkx()
    for edge in graph_nx.edges():
        graph_nx[edge[0]][edge[1]]["neg_min_duration"] = -1 #min([rcpsp_problem.mode_details[edge[1]][mode]["duration"]
                                                            #  for mode in rcpsp_problem.mode_details[edge[1]]])
        if edge[1] == rcpsp_problem.sink_task:
            graph_nx[edge[0]][edge[1]]["neg_min_duration"] = -1
    shortest_path_length = nx.shortest_path_length(graph_nx, source=rcpsp_problem.source_task, weight="neg_min_duration", method="bellman-ford")
    length_to_nodes = {}
    for node, length in shortest_path_length.items():
        if -length not in length_to_nodes:
            length_to_nodes[-length] = []
        length_to_nodes[-length].append(node)
    
    # Determine the maximum layer depth to position the sink task
    max_layer_depth = max(length_to_nodes.keys())
    
    position = {}
    for length in sorted(length_to_nodes.keys()):
        nodes_in_layer = sorted(length_to_nodes[length]) # Sort nodes for consistent y-ordering
        # Distribute y-coordinates evenly for nodes in the same layer
        for i, node in enumerate(nodes_in_layer):
            position[node] = (length, i * 2) # Multiplier '2' for better vertical spacing
    
    # Explicitly ensure the sink task is at the maximum rightmost position
    # If it's already there, this won't change much. If not, it will push it.
    # We'll set its x-coordinate to max_layer_depth + 1 (or any value greater than max_layer_depth)
    # and its y-coordinate to be centered, or whatever works best for single node layer
    sink_task = rcpsp_problem.sink_task
    if sink_task in position: # Ensure the sink task is in the graph
        position[sink_task] = (max_layer_depth + 2, 0) # Push to an even further right layer, centered vertically
    
    # different color for source and sink task
    sink_source_color = "#FFB000"  # Orange
    normal_task_color = "#648FFF"  # Blue
    node_color = [normal_task_color] * len(graph_nx.nodes())
    # Note: Assuming your graph_nx nodes are 1-indexed as in your problem description
    if rcpsp_problem.source_task in graph_nx.nodes():
        node_color[list(graph_nx.nodes()).index(rcpsp_problem.source_task)] = sink_source_color
    if rcpsp_problem.sink_task in graph_nx.nodes():
        node_color[list(graph_nx.nodes()).index(rcpsp_problem.sink_task)] = sink_source_color
    
    # plot
    fig, ax = plt.subplots(1, figsize=(18, 10))
    nx.draw_networkx(
        graph_nx,
        pos=position,
        node_color=node_color,
        node_size=1200,  # Increase node size
        font_size=9,    # Adjust label font size
        with_labels=True,
        width=1.5,       # Make edges slightly thicker
        edge_color="#A3A3A3", # Desaturated edge color for non-critical
        alpha=0.7,       # Slightly transparent edges
        arrows=True,     # Ensure arrows are visible
        arrowsize=15,    # Adjust arrow size
        font_weight='bold' # Make labels bold
    )
    
    if path is not None:
        edges = [(e0, e1) for e0,e1 in zip(path[:-1], path[1:])]
        nx.draw_networkx_edges(
            graph_nx,
            pos=position,
            edgelist=edges,
            edge_color="red", # Red for critical path
            width=10,          # Thicker critical path edges
            ax=ax,
            arrows=True,
            arrowsize=15)
        # Highlight critical path nodes
        critical_nodes_set = set(path) # Use a set for faster lookup
        node_colors_critical = []
        for node in graph_nx.nodes():
            if node in critical_nodes_set:
                node_colors_critical.append("red") # Critical nodes in red
            elif node == rcpsp_problem.source_task or node == rcpsp_problem.sink_task:
                node_colors_critical.append(sink_source_color) # Source/sink color
            else:
                node_colors_critical.append(normal_task_color) # Normal task color
        print(node_colors_critical)
        nx.draw_networkx_nodes(
            graph_nx,
            pos=position,
            node_color=node_colors_critical, # Apply per-node color based on critical path
            node_size=1300,   # Slightly larger for emphasis
            edgecolors="black", # Add a black border
            linewidths=1.5,     # Thicker border for critical nodes
            ax=ax
        )
    
    plt.title("RCPSP Precedence Graph", fontsize=18)
    plt.xlabel("Task Layer/Depth", fontsize=12)
    plt.ylabel("Vertical Spacing (within layer)", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()
    

In [ ]:
plot_predecessors_graph(rcpsp_problem=rcpsp_problem)

# Compute critical path

The critical path in project management is the longest path in the problem that can't be compressed, therefore it is a lower bound on the optimal makespan that is reachable. It represents a path in the precedence graph.
To compute the critical, one can compute a largest path in the precedence constraints. 

In [ ]:
import networkx as nx
graph_nx = rcpsp_problem.graph.to_networkx()
for edge in graph_nx.edges():
    graph_nx[edge[0]][edge[1]]["min_duration"] = min([rcpsp_problem.mode_details[edge[1]][mode]["duration"]
                                                      for mode in rcpsp_problem.mode_details[edge[1]]])
    if edge[1] == rcpsp_problem.sink_task:
        graph_nx[edge[0]][edge[1]]["min_duration"] = 1
path = nx.dag_longest_path(G=graph_nx, weight='min_duration', 
                           default_weight=1, topo_order=None)
edges = [(n1, n2) for n1, n2 in zip(path[:-1], path[1:])]
duration = sum(graph_nx[n[0]][n[1]]["min_duration"] for n in edges)
print("Duration of critical path : ", duration)
print(path)

### Plot the critical path : 

In [ ]:
plot_predecessors_graph(rcpsp_problem, path=path)

## Other procedure to compute critical path or minimum project duration

The critical path can be computed by a graph procedure described in https://www.youtube.com/watch?v=4oDLMs11Exs. It is a quite simple algorithm, doing forward and backward graph exploration. In the end it provides earliest start date, earliest finish date, and their counterpart (for a optimal schedule ignoring resource constraints) : latest start date, latest finish date.

In [ ]:
from discrete_optimization.rcpsp.solvers.cpm import CpmRcpspSolver
solver = CpmRcpspSolver(rcpsp_problem)
critical_path = solver.run_classic_cpm()
edges = [(pi, pi1) for pi, pi1 in zip(critical_path[:-1], critical_path[1:])]
print(solver.map_node[rcpsp_problem.sink_task])

The critical path can be identified as nodes where all the values are equals.

In [ ]:
plot_predecessors_graph(rcpsp_problem, critical_path)

For ressource constrained scheduling problems, this forward/backward pass is not sufficient to compute a schedule, because the ressource capacity constraints are not taken into account. However the *ESD*, *LSD*, *EFD*, *LFD* values can be used in various heuristics to schedule tasks by priority.

## Plotting a solution

In [ ]:
some_solution = rcpsp_problem.get_dummy_solution()

In [ ]:
from discrete_optimization.rcpsp.utils import plot_ressource_view, plot_task_gantt
plot_ressource_view(rcpsp_problem, some_solution)
plot_task_gantt(rcpsp_problem, some_solution)

## SGS : Serial Generation Scheme


The SGS is a "priority-based" constructive heuristic. It takes a pre-defined priority list of tasks (a permutation) and builds a feasible schedule one task at a time.

Here is the core logic for the single-mode case:

1.  Initialize an empty `schedule` dictionary and set the start time of the `source` task to 0.
2.  Maintain a set of `completed_tasks`, initially containing only the `source` task.
3.  Loop until all tasks are scheduled:
    - Find the first task `j` in your `permutation` that is **eligible** to be scheduled. A task is eligible if:
        1. It is not already in `completed_tasks`.
        2. All of its predecessors are in `completed_tasks`.
    - Determine the **earliest possible start time** (`est_j`) for this task `j`. This time must satisfy:
        1. **Precedence:** It must be greater than or equal to the finish times of all its predecessors.
        2. **Resources:** It must be the first time `t >=` (precedence finish time) where there are enough available resources for the entire duration `d_j` of the task.
    - **Schedule the task:** Set `schedule[j] = (est_j, est_j + d_j)`.
    - **Update state:** Add `j` to `completed_tasks` and decrease the available resources for the time window `[est_j, est_j + d_j)`.

## Exercise : 
Now, let's translate this logic into Python!

In [ ]:
import numpy as np
from typing import List, Hashable

def sgs_algorithm(rcpsp_problem: RcpspProblem,
                  permutation_of_task: List[Hashable],
                  predecessors: dict[Hashable, set[Hashable]]):
    # Compute a predecessors mapping for each task. 
    if predecessors is None:
        predecessors = {k: set() for k in rcpsp_problem.tasks_list}
        for k in rcpsp_problem.successors:
            succ = rcpsp_problem.successors[k]
            for s in succ:
                predecessors[s].add(k)

    # Pre-compute durations for each task (assuming mode 1)
    duration_task = {k: rcpsp_problem.mode_details[k][1]["duration"] for k in rcpsp_problem.tasks_list}
    # Pre-compute resource needs for each task
    resource_consumptions = {k: {r: rcpsp_problem.mode_details[k][1].get(r, 0) for r in rcpsp_problem.resources_list}
                             for k in rcpsp_problem.tasks_list}
    # Initialize the schedule dictionary to store start and end times
    schedule = {t: {"start_time": None, "end_time": None} for t in rcpsp_problem.tasks_list}
    # Resource availability is tracked over the project horizon.
    # It's a dictionary of arrays, one for each resource.
    resources_availability = {r: np.array(rcpsp_problem.get_resource_availability_array(r))
                              for r in rcpsp_problem.resources_list}
    
    # Set of tasks that are already scheduled and finished.
    completed_tasks = set()

    # The source task is the first to be scheduled, at time 0.
    source_task = rcpsp_problem.source_task
    schedule[source_task]["start_time"] = 0
    schedule[source_task]["end_time"] = 0
    completed_tasks.add(source_task)

    # Main loop: continue until all tasks are scheduled
    while len(completed_tasks) < rcpsp_problem.n_jobs:
        # 1. Find the next eligible task from the permutation
        next_task = None
        
        # 2. Determine the earliest start time based on PRECEDENCE constraints
        # This is the maximum of the end times of all its predecessors.
        earliest_start_by_pred = 0
        

        # 3. Find the final start time by also considering RESOURCE constraints
        # Starting from earliest_start_by_pred, find the first time slot 't'
        # where all required resources are available for the task's full duration.
        start_of_task = 0
        
        # ... Your implementation here ...
        
        # 4. Schedule the task and update resource availability
        schedule[next_task] = {"start_time": start_of_task, "end_time": start_of_task + duration_task[next_task]}
        
        # ... Your implementation here ...

        # 5. Add the task to the set of completed tasks
        completed_tasks.add(next_task)

    return schedule

If you are blocked, you can retrieve one corrected version of the SGS by decommenting the following cell : 

In [ ]:
#%load correction/nb1_correction.py

## Testing the sgs : 
From the sgs output, it is quite easy to rebuild a RCPSPSolution object and check if it returns a feasible schedule, by calling the ".satisfy()" function.

In [ ]:
tasks_list_permutation = list(rcpsp_problem.tasks_list)
import random
random.shuffle(tasks_list_permutation)
schedule = sgs_algorithm(rcpsp_problem, tasks_list_permutation)
print(schedule)
solution = RcpspSolution(problem=rcpsp_problem, rcpsp_schedule=schedule)
print(rcpsp_problem.satisfy(solution))

Evaluate : 

In [ ]:
rcpsp_problem.evaluate(solution)

### Build a permutation based on critical path computation output :
SGS can be seen as a priority based greedy algorithm, the more the task id is on the left side of the permutation, the more it has chance to be scheduled faster. 
We can therefore build heuristic ordering of the task and run SGS on it. One idea it to reuse output of the CPM algorithm to schedule first the task that have the lowest earliest finish date for example, but you can imagine other rules : 

In [ ]:
# list sorted by EFD ?
perm_efd = sorted(rcpsp_problem.tasks_list, 
                  key=lambda x: solver.map_node[x]._EFD)
sol_efd = sgs_algorithm(rcpsp_problem, perm_efd)
solution_efd = RcpspSolution(problem=rcpsp_problem, rcpsp_schedule=sol_efd)
print("Available fields in CPM output : ", solver.map_node[1].__dict__.keys())

perm_esd = sorted(rcpsp_problem.tasks_list, 
                  key=lambda x: solver.map_node[x]._ESD)
sol_esd = sgs_algorithm(rcpsp_problem, perm_esd)
solution_esd = RcpspSolution(problem=rcpsp_problem, rcpsp_schedule=sol_esd)

# Try different methods ?
# What would be your best results ?
print("EFD ", rcpsp_problem.evaluate(solution_efd))
print("ESD ", rcpsp_problem.evaluate(solution_esd))

Can you find other priority rule to get better results, or use a random search on ordering to get better solutions ?

## Bonus : Improving Solutions with Random Sampling
We can search for better solutions by trying many different priority lists. 

The following code implements a simple Random Sampling (also known as Random-Restart Hill Climbing). 

It generates many random permutations, runs the SGS for each, and keeps track of the best solution found.

A true Local Search algorithm would be slightly different: it would start with one permutation and make small, systematic changes to it (like swapping two adjacent tasks) to explore its "neighborhood" for improvements. 


In [ ]:
from copy import deepcopy
def greedy_local_search(problem: RcpspProblem, n_iteration: int = 1000)->RcpspProblem:
    current_permutation = deepcopy(problem.tasks_list)
    random.shuffle(current_permutation)
    schedule = sgs_algorithm(problem, current_permutation)
    makespan = schedule[problem.sink_task]["end_time"]
    best_makespan = makespan
    best_permutation = deepcopy(current_permutation)
    best_schedule = deepcopy(schedule)
    for i in range(n_iteration):
        random.shuffle(current_permutation)
        schedule = sgs_algorithm(problem, current_permutation)
        makespan = schedule[problem.sink_task]["end_time"]
        if makespan < best_makespan:
            print("Improved from ", best_makespan, " to ", makespan)
            best_makespan = makespan
            best_permutation = deepcopy(current_permutation)
            best_schedule = deepcopy(schedule)
    return RcpspSolution(problem=problem, rcpsp_schedule=best_schedule)

In [ ]:
sol = greedy_local_search(problem=rcpsp_problem, n_iteration=10000)
print(rcpsp_problem.satisfy(sol), rcpsp_problem.evaluate(sol))

In [ ]:
plot_ressource_view(rcpsp_problem, sol)
plot_task_gantt(rcpsp_problem, sol)

## [OPTIONAL] Bonus for those interested 
Going through this [Notebook](https://github.com/airbus/discrete-optimization/blob/master/notebooks/RCPSP%20tutorials/RCPSP-3%20Local%20search.ipynb), is using the "vector" representation of the schedule and run metaheuristics to compute better solutions.
Algorithms used are https://en.wikipedia.org/wiki/Simulated_annealing,
https://en.wikipedia.org/wiki/Hill_climbing